In [1]:
# reformat into jsonl file
import os
import json
os.mkdir('data_2017-09/webpages_tmp')
with open('data_2017-09/webpages_tmp/webpages_tmp.jsonl', 'w') as f:
    for i,file in enumerate(os.listdir('data_2017-09/webpages/')):
        webpage = json.load(open('data_2017-09/webpages/' + file, 'r'))
        f.write(json.dumps(webpage) + '\n')
        if i % 10000 == 0: print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [2]:
# sets up the pyserini sparse index
!python3 -m pyserini.index.lucene --stopwords "data_2017-09/stopwords.txt" --collection JsonCollection --input "data_2017-09/webpages_tmp" --index "data_2017-09/pyserini/pyserini_index" --generator DefaultLuceneDocumentGenerator --threads 10 -verbose

2022-05-07 11:43:17,504 INFO  [main] index.IndexCollection (IndexCollection.java:636) - Setting log level to DEBUG
2022-05-07 11:43:17,506 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-05-07 11:43:17,506 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-05-07 11:43:17,507 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: data_2017-09/webpages_tmp
2022-05-07 11:43:17,507 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-05-07 11:43:17,507 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-05-07 11:43:17,508 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 10
2022-05-07 11:43:17,508 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-05-07 11:43:17,508 INFO  [main] index.IndexCollec

In [3]:
os.remove('data_2017-09/webpages_tmp/webpages_tmp.jsonl')
os.rmdir('data_2017-09/webpages_tmp')

In [1]:
from eval import eval

In [8]:
!python3 -m pyserini.search.lucene --stopwords "data_2017-09/stopwords.txt" --batch-size 50 --threads 10 --hits 10 --index "data_2017-09/pyserini/pyserini_index" --topics "data_2017-09/queries/queries_val.tsv" --output out/bm25_runs/run.val_9_0.99.txt --bm25 --k1 9 --b 0.99

# note MAP = MRR when there is exactly one relevant result
# https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-ran
#!python3 -m pyserini.eval.trec_eval -m P.1 "data_2017-09/queries/relevance_scores.txt" out/bm25_runs/run.val_4_0.9.txt

Setting BM25 parameters: k1=9.0, b=0.99
Using custom stopwords=data_2017-09/stopwords.txt
Running data_2017-09/queries/queries_val.tsv topics, saving to out/bm25_runs/run.val_9_0.99.txt...
100%|█████████████████████████████████████| 15344/15344 [04:43<00:00, 54.14it/s]


In [9]:
ru = eval.load_run('out/bm25_runs/run.val_9_0.99.txt')
rs = eval.load_rel_scores('data_2017-09/queries/relevance_scores.txt')
print('P@1: ', eval.compute_p1(rs,ru))
print('MRR@10: ', eval.compute_mrr10(rs,ru))

P@1:  0.2078006783198539
MRR@10:  0.28396783467713604


In [21]:
!python3 -m pyserini.search.lucene --stopwords "data_2017-09/stopwords.txt" --batch-size 50 --threads 10 --max-passage-hits 10 --index "data_2017-09/pyserini/pyserini_index" --topics "data_2017-09/queries_onlylast/queries_val.tsv" --output out/bm25_runs/run.onlylast.val_4_0.99.txt --bm25 --k1 4 --b 0.99

Setting BM25 parameters: k1=4.0, b=0.99
Using custom stopwords=data_2017-09/stopwords.txt
Running data_2017-09/queries_onlylast/queries_val.tsv topics, saving to out/bm25_runs/run.onlylast.val_4_0.99.txt...
100%|█████████████████████████████████████| 15344/15344 [07:49<00:00, 32.66it/s]


In [22]:
ru = eval.load_run('out/bm25_runs/run.onlylast.val_4_0.99.txt')
rs = eval.load_rel_scores('data_2017-09/queries/relevance_scores.txt')
print('P@1: ', eval.compute_p1(rs,ru))
print('MRR@10: ', eval.compute_mrr10(rs,ru))

P@1:  0.25796682497074813
MRR@10:  0.33859208239241295


In [2]:
ru = eval.load_run('out/semantic_runs/run.val.txt')
rs = eval.load_rel_scores('data_2017-09/queries/relevance_scores.txt')
print('P@1: ', eval.compute_p1(rs,ru))
print('MRR@10: ', eval.compute_mrr10(rs,ru))

P@1:  0.12623826903023982
MRR@10:  0.16250007758577883
